In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, global_max_pool, GATv2Conv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score
import tensorflow as tf

In [89]:
mediapipe_edges = [
    (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21),
    (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24),
    (23, 24), (23, 25), (25, 27), (27, 29), (29, 31), (24, 26), (26, 28),
    (28, 30), (30, 32), (0, 11), (0, 12)
]

In [125]:
class HGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(HGCN, self).__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv1b = GCNConv(hidden_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.pool1 = TopKPooling(hidden_channels, ratio=0.8)

        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv2b = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        self.pool2 = TopKPooling(hidden_channels, ratio=0.6)
        

        self.lin1 = nn.Linear(hidden_channels * 2, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, hidden_channels//2)
        self.lin3 = nn.Linear(hidden_channels//2, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv1b(x, edge_index))
        x = self.bn1(x)
        x, edge_index, _, batch, _,_ = self.pool1(x, edge_index, None, batch)
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv2b(x, edge_index))
        x = self.bn2(x)
        x, edge_index, _, batch, _,_ = self.pool2(x, edge_index, None, batch)
        x = torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return F.log_softmax(x, dim=1)
    



In [ ]:
class ImprovedHGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(ImprovedHGCN, self).__init__()
        
        # GCN Block 1
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv1b = GCNConv(hidden_channels, hidden_channels)
        self.bn1b = nn.BatchNorm1d(hidden_channels)
        self.conv1c = GCNConv(hidden_channels, hidden_channels)
        self.bn1c = nn.BatchNorm1d(hidden_channels)
        self.pool1 = TopKPooling(hidden_channels, ratio=0.9)
        self.res1 = nn.Linear(in_channels, hidden_channels)
        
        # GCN Block 2
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        self.conv2b = GCNConv(hidden_channels, hidden_channels)
        self.bn2b = nn.BatchNorm1d(hidden_channels)
        self.conv2c = GCNConv(hidden_channels, hidden_channels)
        self.bn2c = nn.BatchNorm1d(hidden_channels)
        self.pool2 = TopKPooling(hidden_channels, ratio=0.9)
        self.res2 = nn.Linear(hidden_channels, hidden_channels)
        
        # GCN Block 3
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.bn3 = nn.BatchNorm1d(hidden_channels)
        self.conv3b = GCNConv(hidden_channels, hidden_channels)
        self.bn3b = nn.BatchNorm1d(hidden_channels)
        self.conv3c = GCNConv(hidden_channels, hidden_channels)
        self.bn3c = nn.BatchNorm1d(hidden_channels)
        self.pool3 = TopKPooling(hidden_channels, ratio=0.8)
        self.res3 = nn.Linear(hidden_channels, hidden_channels)
        
        # GCN Block 4
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.bn4 = nn.BatchNorm1d(hidden_channels)
        self.conv4b = GCNConv(hidden_channels, hidden_channels)
        self.bn4b = nn.BatchNorm1d(hidden_channels)
        self.conv4c = GCNConv(hidden_channels, hidden_channels)
        self.bn4c = nn.BatchNorm1d(hidden_channels)
        self.pool4 = TopKPooling(hidden_channels, ratio=0.8)
        self.res4 = nn.Linear(hidden_channels, hidden_channels)

        # Final classifier
        self.lin1 = nn.Linear(hidden_channels * 2 * 4, hidden_channels)  # 4 stages × (mean + max)
        self.lin2 = nn.Linear(hidden_channels, hidden_channels // 2)
        self.lin3 = nn.Linear(hidden_channels // 2, num_classes)

    def gcn_block(self, x, edge_index, convs, bns, res_layer, pool, batch):
        # Residual projection
        res = res_layer(x)
        
        # 3-layer GCN with BN
        x = F.relu(bns[0](convs[0](x, edge_index)) + res)
        x = F.relu(bns[1](convs[1](x, edge_index)))
        x = F.relu(bns[2](convs[2](x, edge_index)))
        
        # Pooling
        x, edge_index, _, batch, _, _ = pool(x, edge_index, batch=batch)
        return x, edge_index, batch

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        multi_scale_features = []

        # GCN Block 1
        x, edge_index, batch = self.gcn_block(
            x, edge_index,
            [self.conv1, self.conv1b, self.conv1c],
            [self.bn1, self.bn1b, self.bn1c],
            self.res1, self.pool1, batch)
        multi_scale_features.append(torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1))
        
        # GCN Block 2
        x, edge_index, batch = self.gcn_block(
            x, edge_index,
            [self.conv2, self.conv2b, self.conv2c],
            [self.bn2, self.bn2b, self.bn2c],
            self.res2, self.pool2, batch)
        multi_scale_features.append(torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1))
        
        # GCN Block 3
        x, edge_index, batch = self.gcn_block(
            x, edge_index,
            [self.conv3, self.conv3b, self.conv3c],
            [self.bn3, self.bn3b, self.bn3c],
            self.res3, self.pool3, batch)
        multi_scale_features.append(torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1))

        # GCN Block 4
        x, edge_index, batch = self.gcn_block(
            x, edge_index,
            [self.conv4, self.conv4b, self.conv4c],
            [self.bn4, self.bn4b, self.bn4c],
            self.res4, self.pool4, batch)
        multi_scale_features.append(torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1))

        # Multi-scale feature fusion
        x = torch.cat(multi_scale_features, dim=1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return F.log_softmax(x, dim=1)


Run for LE21

In [179]:
# Load the dataset
import pickle
train_file = r"C:\Users\johna\Desktop\CS3264proj\train_falls_le2.pkl"
test_file = r"C:\Users\johna\Desktop\CS3264proj\test_falls_le2.pkl"
with open(train_file, 'rb') as f:
    dataset = pickle.load(f)
print(f"Loaded dataset size: {len(dataset)}")
with open(test_file, 'rb') as f:
    test_dataset = pickle.load(f)
print(f"Loaded test dataset size: {len(test_dataset)}")

def build_graph(sequence_tensor, label):
    """
    sequence_tensor: Tensor of shape (T, 33, 4)
    label: int (class label for the whole sequence)
    """
    T, J, F = sequence_tensor.shape  # Time, Joints, Features
    x = sequence_tensor.reshape(T * J, F)  # (T*33, 4)


    edge_list = []

    # Intra-frame edges
    for t in range(T):
        for u, v in mediapipe_edges:
            edge_list += [(t * J + u, t * J + v), (t * J + v, t * J + u)]

    # Inter-frame (temporal) self-links
    for t in range(T - 1):
        for j in range(J):
            edge_list += [(t * J + j, (t + 1) * J + j), ((t + 1) * J + j, t * J + j)]

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index, y=torch.tensor(label, dtype=torch.long))
    # data = Data(x=x, edge_index=edge_index, y=label)
    print(f"Data object: {data}")
    # print(f"Data x shape: {data.x.shape}")
    # print(f"Data edge_index shape: {data.edge_index.shape}")
    # print(f"Data y shape: {data.y}")
    # print(f"Data num_nodes: {data.num_nodes}")
    return data
graphs = []
for sequence_tensor, label in dataset:
    # Convert TensorFlow tensor to NumPy, then to PyTorch tensor
    sequence_tensor = torch.tensor(sequence_tensor.numpy())  # Convert to PyTorch tensor
    sequence_tensor = sequence_tensor.permute(1,2,0)  # Change shape to (T, 4, 33)
    graph = build_graph(sequence_tensor, label)
    graphs.append(graph)

test_graphs = []
for sequence_tensor, label in test_dataset:
    sequence_tensor = torch.tensor(sequence_tensor.numpy())  # Convert to PyTorch tensor
    sequence_tensor = sequence_tensor.permute(1,2,0)  # Change shape to (T, 4, 33)
    graph = build_graph(sequence_tensor, label)
    test_graphs.append(graph)

Loaded dataset size: 152
Loaded test dataset size: 38
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=1)
Data object: Data(

In [ ]:
model = HGCN(in_channels=4, hidden_channels=64, num_classes=2)
# model = ImprovedHGCN(in_channels=4, hidden_channels=64, num_classes=2)
train_loader = DataLoader(graphs, batch_size=16, shuffle=True) #batch default is 32

test_loader = DataLoader(test_graphs, batch_size=16, shuffle=False)

In [160]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01) #try sgd or adam, or adamw
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Training loop
epochs = 50 # May need more epochs for the combined model to train
overall_acc = 0.0
overall_loss = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, batch in enumerate(train_loader):
        batch_data = batch.to(device)  # Move the entire batch to the device
        batch_labels = batch.y.to(device)  # Extract labels
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

        loss_per_epoch = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss_per_epoch:.4f}, Accuracy: {epoch_acc:.2f}%")
        
    
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        for i, batch_test_data in enumerate(test_loader):
            batch_test_data = batch_test_data.to(device)  # Move the entire batch to the device
            batch_test_labels = batch_test_data.y.to(device)  # Extract labels
            optimizer.zero_grad()
            # Forward pass

            outputs = model(batch_test_data)
            loss = criterion(outputs, batch_test_labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += batch_test_labels.size(0)
            correct += (predicted == batch_test_labels).sum().item()
            
            all_preds.append(predicted.cpu())
            all_labels.append(batch_test_labels.cpu())


        val_loss /= len(test_loader)
        val_acc = 100 * correct / total
        
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        val_f1 = f1_score(all_labels.cpu(), all_preds.cpu(), average='weighted')

        print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%, Val F1 Score: {val_f1:.4f}")

        # Save model if it's the best so far
        if val_loss < overall_loss or val_acc > overall_acc:
            overall_acc = val_acc
            overall_loss = val_loss
            # torch.save(model.state_dict(), 'best_stgcn_reasoning_model.pth')
            print(f"Saved new best model with Val Accuracy: {overall_acc:.2f}% and F1 Score: {val_f1:.4f}")

    scheduler.step() # Step the scheduler after validation

print("Training completed.")
print(f"Best Val Accuracy achieved: {overall_acc:.2f}%")
print(f"Best Val Loss achieved: {overall_loss:.4f}")


Epoch [1/50], Step [1/10], Loss: 0.0739, Accuracy: 43.75%
Epoch [1/50], Step [2/10], Loss: 0.1467, Accuracy: 40.62%
Epoch [1/50], Step [3/10], Loss: 0.2137, Accuracy: 45.83%
Epoch [1/50], Step [4/10], Loss: 0.2734, Accuracy: 53.12%
Epoch [1/50], Step [5/10], Loss: 0.3297, Accuracy: 58.75%
Epoch [1/50], Step [6/10], Loss: 0.3772, Accuracy: 63.54%
Epoch [1/50], Step [7/10], Loss: 0.4376, Accuracy: 65.18%
Epoch [1/50], Step [8/10], Loss: 0.4830, Accuracy: 66.41%
Epoch [1/50], Step [9/10], Loss: 0.5424, Accuracy: 67.36%
Epoch [1/50], Step [10/10], Loss: 0.6086, Accuracy: 68.42%
Epoch [1/50], Val Loss: 0.7354, Val Accuracy: 31.58%, Val F1 Score: 0.1516
Saved new best model with Val Accuracy: 31.58% and F1 Score: 0.1516
Epoch [2/50], Step [1/10], Loss: 0.0601, Accuracy: 68.75%
Epoch [2/50], Step [2/10], Loss: 0.1031, Accuracy: 81.25%
Epoch [2/50], Step [3/10], Loss: 0.1334, Accuracy: 83.33%
Epoch [2/50], Step [4/10], Loss: 0.2011, Accuracy: 78.12%
Epoch [2/50], Step [5/10], Loss: 0.2458, Acc

In [176]:
import pickle
file = r"C:\Users\johna\Desktop\CS3264proj\pose_dataset.pkl"
with open(file, 'rb') as f:
    dataset = pickle.load(f)
print(f"Loaded dataset size: {len(dataset)}")
print(dataset[0][1])

Loaded dataset size: 190
1


Run for URfall

In [181]:
import pickle
import pickletools
train_file = r"C:\Users\johna\Desktop\CS3264proj\train_ur_fall_50_frames_4_features.pkl"
val_file = r'C:\Users\johna\Desktop\CS3264proj\val_ur_fall_50_frames_4_features.pkl'  

with open('train_ur_fall_50_frames_4_features.pkl', 'rb') as f:
    dataset = pickle.load(f)
print(f"Loaded dataset size: {len(dataset)}")
with open('val_ur_fall_50_frames_4_features.pkl', 'rb') as f:
    test_dataset = pickle.load(f)
print(f"Loaded test dataset size: {len(test_dataset)}")

def build_graph(sequence_tensor, label):
    """
    sequence_tensor: Tensor of shape (T, 33, 4)
    label: int (class label for the whole sequence)
    """
    T, J, F = sequence_tensor.shape  # Time, Joints, Features
    x = sequence_tensor.reshape(T * J, F)  # (T*33, 4)


    edge_list = []

    # Intra-frame edges
    for t in range(T):
        for u, v in mediapipe_edges:
            edge_list += [(t * J + u, t * J + v), (t * J + v, t * J + u)]

    # Inter-frame (temporal) self-links
    for t in range(T - 1):
        for j in range(J):
            edge_list += [(t * J + j, (t + 1) * J + j), ((t + 1) * J + j, t * J + j)]

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index, y=torch.tensor(label, dtype=torch.long))
    # data = Data(x=x, edge_index=edge_index, y=label)
    print(f"Data object: {data}")
    # print(f"Data x shape: {data.x.shape}")
    # print(f"Data edge_index shape: {data.edge_index.shape}")
    # print(f"Data y shape: {data.y}")
    # print(f"Data num_nodes: {data.num_nodes}")
    return data

graphs = []
for sequence_tensor, label in dataset:
    # Convert TensorFlow tensor to NumPy, then to PyTorch tensor
    sequence_tensor = torch.tensor(sequence_tensor.numpy())  # Convert to PyTorch tensor
    graph = build_graph(sequence_tensor, label)
    graphs.append(graph)

test_graphs = []
for sequence_tensor, label in test_dataset:
    sequence_tensor = torch.tensor(sequence_tensor.numpy())  # Convert to PyTorch tensor
    graph = build_graph(sequence_tensor, label)
    test_graphs.append(graph)


Loaded dataset size: 60
Loaded test dataset size: 12
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x=[1650, 4], edge_index=[2, 6534], y=0)
Data object: Data(x

In [171]:
model = HGCN(in_channels=4, hidden_channels=64, num_classes=2)
# model = ImprovedHGCN(in_channels=4, hidden_channels=64, num_classes=2)
# Create a DataLoader for the dataset
train_loader = DataLoader(graphs, batch_size=16, shuffle=True) #batch default is 32

test_loader = DataLoader(test_graphs, batch_size=16, shuffle=False)

In [180]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01) #try sgd or adam, or adamw
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Training loop
epochs = 50 # May need more epochs for the combined model to train
overall_acc = 0.0
overall_loss = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, batch in enumerate(train_loader):
        batch_data = batch.to(device)  # Move the entire batch to the device
        batch_labels = batch.y.to(device)  # Extract labels
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

        loss_per_epoch = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss_per_epoch:.4f}, Accuracy: {epoch_acc:.2f}%")
        
    
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        for i, batch_test_data in enumerate(test_loader):
            batch_test_data = batch_test_data.to(device)  # Move the entire batch to the device
            batch_test_labels = batch_test_data.y.to(device)  # Extract labels
            optimizer.zero_grad()
            # Forward pass

            outputs = model(batch_test_data)
            loss = criterion(outputs, batch_test_labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += batch_test_labels.size(0)
            correct += (predicted == batch_test_labels).sum().item()
            
            all_preds.append(predicted.cpu())
            all_labels.append(batch_test_labels.cpu())


        val_loss /= len(test_loader)
        val_acc = 100 * correct / total
        
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        val_f1 = f1_score(all_labels.cpu(), all_preds.cpu(), average='weighted')

        print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%, Val F1 Score: {val_f1:.4f}")

        # Save model if it's the best so far
        if val_loss < overall_loss or val_acc > overall_acc:
            overall_acc = val_acc
            overall_loss = val_loss
            # torch.save(model.state_dict(), 'best_stgcn_reasoning_model.pth')
            print(f"Saved new best model with Val Accuracy: {overall_acc:.2f}% and F1 Score: {val_f1:.4f}")

    scheduler.step() # Step the scheduler after validation

print("Training completed.")
print(f"Best Val Accuracy achieved: {overall_acc:.2f}%")
print(f"Best Val Loss achieved: {overall_loss:.4f}")


Epoch [1/50], Step [1/4], Loss: 0.0069, Accuracy: 100.00%
Epoch [1/50], Step [2/4], Loss: 0.1056, Accuracy: 93.75%
Epoch [1/50], Step [3/4], Loss: 0.1264, Accuracy: 95.83%
Epoch [1/50], Step [4/4], Loss: 0.1801, Accuracy: 93.33%
Epoch [1/50], Val Loss: 0.5792, Val Accuracy: 83.33%, Val F1 Score: 0.8333
Saved new best model with Val Accuracy: 83.33% and F1 Score: 0.8333
Epoch [2/50], Step [1/4], Loss: 0.0156, Accuracy: 100.00%
Epoch [2/50], Step [2/4], Loss: 0.0278, Accuracy: 100.00%
Epoch [2/50], Step [3/4], Loss: 0.0833, Accuracy: 97.92%
Epoch [2/50], Step [4/4], Loss: 0.0851, Accuracy: 98.33%
Epoch [2/50], Val Loss: 0.5734, Val Accuracy: 91.67%, Val F1 Score: 0.9172
Saved new best model with Val Accuracy: 91.67% and F1 Score: 0.9172
Epoch [3/50], Step [1/4], Loss: 0.0046, Accuracy: 100.00%
Epoch [3/50], Step [2/4], Loss: 0.0258, Accuracy: 100.00%
Epoch [3/50], Step [3/4], Loss: 0.0339, Accuracy: 100.00%
Epoch [3/50], Step [4/4], Loss: 0.0441, Accuracy: 100.00%
Epoch [3/50], Val Loss: